# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The **launch success rate** may depend on many factors such as **payload mass**, **target destination orbit**, etc.

It may also depend on the **location** and **proximities** of a launch site, i.e., the initial position of rocket trajectories.

Finding an **optimal location** for building a **launch site** certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous **Exploratory Data Analysis labs**, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary **correlations** between the **launch site** and **success rates**.

In this lab, we will be performing more **interactive visual analytics** using `Folium`.

---

## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
# !pip3 install folium
# !pip3 install wget

In [6]:
# import folium
# import wget
# import pandas as pd

In [1]:
# for data manipulation
import pandas as pd
# for mapping
import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [2]:
# local path for geo csv file
local_path = r"D:\IBM Professional Certification\10_Data Science Capstone Project\5_EDA with folium\data\spacex_launch_geo.csv"

In [3]:
# # Download and read the `spacex_launch_geo.csv`
# spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

# load geo csv dataset
spacex_df = pd.read_csv(local_path)

Now, you can take a look at what are the coordinates for each site.


In [4]:
# make a copy & check original dataset
spacex_df_copy = spacex_df.copy()
spacex_df_copy.head(3)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`

## columns relevant for eda 
cols_retain = [
    'Launch Site',
    'Lat', 'Long',
    'class'
]

# truncate original dataframe to retain relevant columns only
spacex_df = spacex_df[cols_retain]

# group data by launch site
# launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).apply(func=lambda value: value.loc[value.index[0], :])

# truncate grouped dataset
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# view final grouped dataset
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


#### Note on the used methodology:
```python
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
```
This performs same action as $\rightarrow$
```python
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).apply(func=lambda value: value.loc[value.index[0], :])
```
However, the later is used since `.first()` has been deprecated by `pandas`.

#### Note
Above **coordinates** are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by **pinning them on a map**.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
    ## nasa latitude & longitude
nasa_lat, nasa_lon = 29.559684888503615, -95.0830971930759
    ## coordinate list for folium `location` argument
nasa_coordinate = [nasa_lat, nasa_lon]

    ## instantiate folium map object
site_map = folium.Map(location=nasa_coordinate,
                      zoom_start=10)

# site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
# when one clicks on the circle only then will
# a popup will appear with the message "NASA Johnson Space Center" (full form name)

    ## popup child object for circle marker
popup_johnson = folium.Popup(html="NASA Johnson Space Center",
                                parse_html=False, show=False)

    ## circle marker child object    
circle = folium.Circle(nasa_coordinate,
                       radius=1000,
                       color='#A0153E',
                       fill=False,
                      fill_opacity=0.4,
                      fill_color="#FEECE2",
                      opacity=1)

    ## add popup child to circle marker child
circle.add_child(popup_johnson)



# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
# always shows a marker with icon saying 'NASA JSC' (short form name) at center of the circle created above
# to indicate visually the location of Johnson Space Center

    ## icon for map marker
# icon_johnson = DivIcon(
#                 icon_size=(20, 20),
#                 icon_anchor=(0, 0),
#                 html='<div style="font-size: 14; color:#5D0E41;"><b>%s</b></div>' % 'NASA JSC')
icon_johnson = DivIcon(
                icon_size=(20, 20),
                icon_anchor=(0, 0),
                html='<div style="font-size: 14; color:#481E14;"><b>NASA JSC</b></div>')


    ## create marker
marker = folium.map.Marker(
                location=nasa_coordinate,
                icon=icon_johnson)


# add both circle child & marker child to the original map
site_map.add_child(circle)
site_map.add_child(marker)

site_map

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# DATA GENERATION

sites = launch_sites_df["Launch Site"]
lats = launch_sites_df["Lat"]
lons = launch_sites_df["Long"]

sites_full = {
    "CCAFS LC-40":"Cape Canaveral Launch Complex 40",
    "CCAFS SLC-40":"Cape Canaveral Space Launch Complex 40",
    "KSC LC-39A":"Kennedy Space Center Launch Complex 39A",
    "VAFB SLC-4E":"Vandenberg AFB Space Launch Complex 4"
}

In [105]:
# MAP GENERATION

# Initial the map
site_map = folium.Map(location=nasa_coordinate,
                      zoom_start=3.15)

# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for site, lat, lon in zip(sites, lats, lons):
    # create a circle child object
    circle = folium.Circle(location=[lat, lon],
                 radius=1000,
                 color="#481E14",
                 fill=False,
                 fill_opacity=0.04,
                 fill_color="#FBF3D5",
                 opacity=0.5)
    # create a popup child object for the circle to show on clicking
    popup = folium.Popup(html=sites_full[site],
                        parse_html=False,
                        show=False)
    
    
    # add the popup child to circle child
    circle.add_child(popup)
    
    
    # create an icon object for the `icon` argument of folium marker
    icon = DivIcon(icon_size=(20, 20),
                  icon_anchor=(0, 0),
                  html=f'<div style="font-size: 14; color:#A0153E;"><b>{str(site)}</b></div>')
    
    # create a marker child object for locating the launch site
    marker = folium.map.Marker(location=[lat, lon],
                              icon=icon)
    
    # add circle and marker child objects to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# create a polyline child object to render line of equator
    ## equator coordinates for polyline -> [[lat_start, lon_start], [lat_end, lon_end]]
equator_coordinates = [
    [0, -180],
    [0, 180]
]
    ## create polyline object
equator_line = folium.PolyLine(locations=equator_coordinates,
                              color="#35374B",
                              weight=2.2,
                              tooltip="Equator",
                              no_clip=True)
    ## marker for equator line
equator_marker = folium.map.Marker(location=[0, -98],
                                  icon=DivIcon(icon_size=(20, 20),
                                              icon_anchor=(0, 0),
                                              html='<div style="font-size:14; color:#35374B"><b>Equator</b></div>'))

    ## add equator child to the map
site_map.add_child(equator_line)

    ## add marker to the map
site_map.add_child(equator_marker)

site_map ## render map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, we can explore the map by zoom-in/out the marked areas, and try to answer the following questions:
1. Are all launch sites in proximity to the Equator line?
2. Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

#### Observations

##### Proximity to Equator line
1. From the map it seems that locations of launch sites were chosen to be as **close to line of equator** as possible (_while remaining in mainland USA_).
2. The choice of location has strategic significance as per [NASA website](https://science.nasa.gov/learn/basics-of-space-flight/chapter14-1/#:~:text=If%20a%20spacecraft,km%20per%20hour) $\rightarrow$
    - allows rockets to get an **extra boost** from Earth's rotational motion
    - at the Equator the **rotational speed** is highest ($1650\;km/hour$ relative to Earth's center)
    - the closer the rocket is to the Equator when launching the greater initial boost it can receive from Earth's rotational motion. Hence it will require relatively **lesser fuel per unit distance** that otherwise or a given rocket can **launch heavier payloads** for same amount of fuel.
    
##### Proximity to the coast
1. All the launch sites are located **along or near the coastline**
2. This again has strategic significance primarily in terms of **safety**
    - in the event of a failed launch even if the rocket falls back it will do so over the ocean and not on the mainland (_where human population exists_)
3. However, contrary to what is preferred, Location of **Vandenberg AFB Space Launch Complex 4** is on the **west coast** of USA
    - further research may be needed to explain this peculiar choice of loaction
    - launching from here will require **launching rockets over populated areas inland** (_towards east_) of having to launch towards west by going **against Earth's rotational motion** (_requires greater fuel per unit distance_)
    
---

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [106]:
# check last ten rows of dataset
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,#41B06E
47,KSC LC-39A,28.573255,-80.646895,1,#41B06E
48,KSC LC-39A,28.573255,-80.646895,1,#41B06E
49,CCAFS SLC-40,28.563197,-80.576820,1,#41B06E
50,CCAFS SLC-40,28.563197,-80.576820,1,#41B06E
51,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38
52,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38
53,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38
54,CCAFS SLC-40,28.563197,-80.576820,1,#41B06E
55,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [107]:
# marker cluster object
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [108]:
# #41B06E -> Greenish color (Successful landing)
# #DF2E38 -> Reddish color (Failed landing)
spacex_df["marker_color"] = spacex_df["class"].apply(func=lambda value: "#41B06E" if value == 1 else "#DF2E38")

# check last ten rows
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,#41B06E
47,KSC LC-39A,28.573255,-80.646895,1,#41B06E
48,KSC LC-39A,28.573255,-80.646895,1,#41B06E
49,CCAFS SLC-40,28.563197,-80.576820,1,#41B06E
50,CCAFS SLC-40,28.563197,-80.576820,1,#41B06E
51,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38
52,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38
53,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38
54,CCAFS SLC-40,28.563197,-80.576820,1,#41B06E
55,CCAFS SLC-40,28.563197,-80.576820,0,#DF2E38


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [109]:
# DATA RETRIEVAL

sites = spacex_df["Launch Site"]
lats = spacex_df["Lat"]
lons = spacex_df["Long"]
colors = spacex_df["marker_color"]

In [110]:
# create marker cluster object
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# add markers to the marker cluster
for site, lat, lon, color in zip(sites, lats, lons, colors):
    # TODO: Create and add a Marker cluster to the site map
        ## custom icon for marker
    icon = folium.Icon(icon_color=color,
                      color="white",
                      icon="glyphicon glyphicon-plus-sign" if color == "#41B06E" else "glyphicon glyphicon-minus-sign")
        ## marker for the marker cluster
    marker = folium.map.Marker(location=[lat, lon],
                              icon=icon)
        ## add marker child to the marker cluster
    marker_cluster.add_child(marker)
    
# render map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [111]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

    ## create a mouse position child object
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=5,
    prefix="Lat: ")

    ## add mouse position child object to the map
site_map.add_child(mouse_position)

# render map
site_map

#### Self Note on above Code
```python
    ## define a custom JavaScript function
    ## for formatting the coordinates to 5 decimal places
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

    ## create a mouse position child object
mouse_position = MousePosition(
    position="topright",
    separator=" Long: ",
    empty_string="NaN",
    lng_first=False,
    num_digits=20,
    prefix="Lat: ",
    lat_formatter=formatter,
    lng_formatter=formatter,
)
```
This will achieve same effect as the one used above. To avoid redundancy the shorter approach has been taken.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [112]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [113]:
# DATA RETRIEVAL 

spacex_df[spacex_df["Launch Site"] == "CCAFS LC-40"].loc[0,["Lat", "Long"]]

Lat     28.562302
Long   -80.577356
Name: 0, dtype: object

In [114]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

## corrdinates of closest coastline to Cape Canaveral Space Launch Complex 40
coast_lat, coast_lon = 28.55656, -80.56689
## coordinates of Cape Canaveral Space Launch Complex 40
ccslc_lat, ccslc_lon = 28.562302, -80.577356
## distance from coastline
distance_coastline = calculate_distance(lat1=ccslc_lat, lon1=ccslc_lon,
                                       lat2=coast_lat, lon2=coast_lon)

distance_coastline

1.20556358178415

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [115]:
# Create and add a folium.Marker on your selected closest coastline point on the map

    ## custom icon for distance marker
distance_icon = DivIcon(icon_size=(20, 20),
                       icon_anchor=(0, 0),
                       html=f'<div style="font-size:14; color:#35374B"><b>Coastline distance: {round(distance_coastline,2)} km</b></div>')
    ## marker child object to pinpoint coastline
distance_marker = folium.map.Marker(location=[coast_lat, coast_lon],
                                   icon=distance_icon)

    ## add distance marker child to the map
site_map.add_child(distance_marker)

# render map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [116]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

    ## location coordinates for drawing poly line
line_coordinates = [
    [ccslc_lat, ccslc_lon],
    [coast_lat, coast_lon]
]

    ## Polyline child object
line = folium.PolyLine(locations=line_coordinates,
                      weight=2,
                      color="red")
    ## add polyline child to map
site_map.add_child(line)

# render map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [118]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

    ## coordinates
ksclc_lat, ksclc_lon = 28.57335, -80.64688 ## KSC LC-39A
kennedy_hw_lat, kennedy_hw_lon = 28.57134, -80.65542 ## Kennedy Parkway
titu_lat, titu_lon = 28.60924, -80.80822 ## Titusville city
rail_lat, rail_lon = 28.57474, -80.6542 ## Railway track

    ## distances
distance_hw = calculate_distance(lat1=ksclc_lat, lon1=ksclc_lon,
                                  lat2=kennedy_hw_lat, lon2=kennedy_hw_lon) ## KSC LC 39A to Kennedy Parkway
distance_titu = calculate_distance(lat1=ksclc_lat, lon1=ksclc_lon,
                                  lat2=titu_lat, lon2=titu_lon) ## KSC LC 39A to Titusville city
distance_rail = calculate_distance(lat1=ksclc_lat, lon1=ksclc_lon,
                                  lat2=rail_lat, lon2=rail_lon) ## KSC LC 39A to Railway line

    ## lines & markers
line_hw = folium.PolyLine(locations=[[ksclc_lat, ksclc_lon], [kennedy_hw_lat, kennedy_hw_lon]],
                              weight=2, color="blue")
distance_marker_hw = folium.map.Marker(location=[kennedy_hw_lat, kennedy_hw_lon],
                                           icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0),
                                                       html=f'<div style="font-size:14; color:#35374B"><b>{round(distance_hw,2)} km</b></div>'))

line_titu = folium.PolyLine(locations=[[ksclc_lat, ksclc_lon], [titu_lat, titu_lon]],
                              weight=2, color="blue")
distance_marker_titu = folium.map.Marker(location=[titu_lat, titu_lon],
                                           icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0),
                                                       html=f'<div style="font-size:14; color:#35374B"><b>{round(distance_titu,2)} km</b></div>'))

line_rail = folium.PolyLine(locations=[[ksclc_lat, ksclc_lon], [rail_lat, rail_lon]],
                              weight=2, color="blue")
distance_marker_rail = folium.map.Marker(location=[rail_lat, rail_lon],
                                           icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0),
                                                       html=f'<div style="font-size:14; color:#35374B"><b>{round(distance_rail,2)} km</b></div>'))

    ## add lines & markers to map
site_map.add_child(line_hw)
site_map.add_child(distance_marker_hw)

site_map.add_child(line_titu)
site_map.add_child(distance_marker_titu)

site_map.add_child(line_rail)
site_map.add_child(distance_marker_rail)

# render map
site_map

After we plot distance lines to the proximities, you can answer the following questions easily:
1. Are launch sites in close proximity to railways?
2. Are launch sites in close proximity to highways?
3. Are launch sites in close proximity to coastline?
4. Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

#### Observations

##### Proximity to Railways
1. The launch site location is in close proximity to railways
2. **Example**: *Nearest railway line to the **KSC LC-39A** center is $\approx 0.73\;km$ only*

##### Proximity to Highways
1. The launch site location is in close proximity to highways
2. **Example**: *Nearest highway to the **KSC LC-39A** center, the **Kennedy Parkway**, is $\approx 0.86\;km$ only*

##### Proximity to Coastline
1. The launch site location is in close proximity to coastlines
2. **Example**: _**CCAFS SLC-40** center is located at a distance of $\approx1.21\;km$ only from the coastline_

##### Proximity to Citites
1. Compared to railway lines, highways, and coastlines, the launch cites seem to be at a larger distance away from the cities
2. **Example**: *Nearest city, **Titusville**, is $\approx16.26\;km$ from the **KSC LC-39A** center*
    - this is  $\approx13.5$ times the distance between CCAFS SLC-40 and the coastline, and $\approx22.27$ times the distance between KSC LC-39A and the railways

##### Strategic significance of the choice of Launch sites
1. **Logistics efficiency**
    - transporting equipments, human resource and other resources to-and-fro the hinterland is easier when launch site is close to railways and highways
2. **Fuel efficiency**
    - closer to the site is to the eastern coastline, the greater boost the rocket can receive from Earth's rotational motion (_Earth rotates from west to east_)
3. **Safety concerns**
    - sites are closer to coastline and far from the cities, hence in event of failure to launch properly, space debris will fall over the oceans and far away from human population